# IMPORTS

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets,models,transforms
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import math
import os
import copy
import time
import sqlite3 as sql
import cv2
from tqdm import tqdm

# CONSTANTS DEFINITION

In [3]:
MLP_KERNEL_SIZE = 1
MLP_FILTERS_1 = 256
MLP_FILTERS_2 = 64
CONTRACTION_RATIO = 2
PADDING_METHOD = 'same'
UPSAMPLE_MODE = 'nearest'
N_FEATURE_EXTRACTED = 2048
N_CLASSES = 17
SEQUENCE_SIZE = 112
BATCH_SIZE = 256
BATCH_PER_EPOCH = 1000

# DATA LOADING

In [8]:
db = "/scratch/users/mdelabrassinne/Database/SoccerDB.db"
con = sql.connect(db)
cur = con.cursor()
query_result = cur.execute("SELECT Path_video FROM VIDEO WHERE Path_video LIKE '%england_epl%' OR Path_video LIKE '%france_ligue-1%' OR Path_video LIKE '%europe_uefa-champions-league%'")
training_vids = copy.deepcopy([q[0] for q in query_result.fetchall()])

# Extract the frames of a part of a video

In [10]:
def extract_frames(video_path, start_time, end_time, frame_rate ,frames):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Set the starting frame to extract
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

    # Calculate the total number of frames to extract
    total_frames = int((end_time - start_time) * frame_rate)

    # Extract the frames
    for i in tqdm(range(total_frames)):
        # Read the next frame
        ret, frame = cap.read()
        if ret:
            # Add the frame to the list
            frames.append(frame)
        else:
            break

        # Set the position to the next frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(cap.get(cv2.CAP_PROP_POS_FRAMES)) + int(cap.get(cv2.CAP_PROP_FPS) / frame_rate))

    # Release the video capture
    cap.release()

# test of the function:
frames = []

%timeit extract_frames('/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv',0,112,1,frames)

i = 0
for i in range(5):
    cv2.imwrite("frame{i}.png".format(i=i),frames[i])

100%|██████████| 112/112 [00:07<00:00, 15.03it/s]

7.57 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
torchvision.io.read_video('/scratch/users/mdelabrassinne/Database/england_epl/2014-2015/2015-02-21_-_18-00_Chelsea_1_-_1_Burnley/1_224p.mkv',0,112,)

In [7]:
for i in range(3):
    print("frame{i}".format(i = i))

frame0
frame1
frame2


In [13]:
def batch_creation(batch_size,sequence_size,videos):
    
    #get the videos path we consider
    videos_chosen = np.random.choice(len(videos),256)
    
    #get the associated jsons
    jsons = [os.path.join(os.path.dirname(videos[vid]),"Labels-v2.json") for vid in videos_chosen]
    
    #zip the 2 
    result =  [(videos[vid],json) for vid,json in zip(videos_chosen,jsons)]
    starting_times = np.random.choice(45*60,256)
    images = []
    for i in tqdm(len(starting_times.shape[0])):
        X = []
        extract_frames(videos[videos_chosen[i]],starting_times[i],starting_times[i]+112,1,X)
        get_actions = 
        
        
        
#test
batch_creation(BATCH_SIZE,SEQUENCE_SIZE,training_vids)


[('/scratch/users/mdelabrassinne/Database/france_ligue-1/2014-2015/2015-04-05_-_22-00_Marseille_2_-_3_Paris_SG/1_224p.mkv', '/scratch/users/mdelabrassinne/Database/france_ligue-1/2014-2015/2015-04-05_-_22-00_Marseille_2_-_3_Paris_SG/Labels-v2.json'), ('/scratch/users/mdelabrassinne/Database/europe_uefa-champions-league/2014-2015/2015-03-18_-_22-45_Dortmund_0_-_3_Juventus/1_224p.mkv', '/scratch/users/mdelabrassinne/Database/europe_uefa-champions-league/2014-2015/2015-03-18_-_22-45_Dortmund_0_-_3_Juventus/Labels-v2.json'), ('/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2015-09-26_-_17-00_Leicester_2_-_5_Arsenal/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2015-09-26_-_17-00_Leicester_2_-_5_Arsenal/Labels-v2.json'), ('/scratch/users/mdelabrassinne/Database/france_ligue-1/2015-2016/2015-11-07_-_19-00_Paris_SG_5_-_0_Toulouse/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/france_ligue-1/2015-2016/2015-11-07_-_19-00_Paris_SG_5_-_0_Toulouse/L

In [12]:
import json
json.load(open('/scratch/users/mdelabrassinne/Database/europe_uefa-champions-league/2015-2016/2015-09-16_-_21-45_Chelsea_4_-_0_Maccabi_Tel_Aviv/Labels-v2.json'))

{'UrlLocal': 'europe_uefa-champions-league/2015-2016/2015-09-16 - 21-45 Chelsea 4 - 0 Maccabi Tel Aviv/',
 'UrlYoutube': '',
 'annotations': [{'gameTime': '1 - 00:00',
   'label': 'Kick-off',
   'position': '165',
   'team': 'home',
   'visibility': 'visible'},
  {'gameTime': '1 - 00:34',
   'label': 'Ball out of play',
   'position': '34658',
   'team': 'not applicable',
   'visibility': 'visible'},
  {'gameTime': '1 - 00:41',
   'label': 'Throw-in',
   'position': '41913',
   'team': 'away',
   'visibility': 'not shown'},
  {'gameTime': '1 - 00:44',
   'label': 'Foul',
   'position': '44097',
   'team': 'home',
   'visibility': 'visible'},
  {'gameTime': '1 - 00:45',
   'label': 'Ball out of play',
   'position': '45700',
   'team': 'not applicable',
   'visibility': 'visible'},
  {'gameTime': '1 - 00:59',
   'label': 'Yellow card',
   'position': '59547',
   'team': 'home',
   'visibility': 'visible'},
  {'gameTime': '1 - 01:49',
   'label': 'Indirect free-kick',
   'position': '109

In [ ]:
class VideoFrameDataset(torch.utils.data.Dataset):
    
    def __init__(self,data_dir,transform=None):
        db = "/scratch/users/mdelabrassinne/Database/SoccerDB.db"
        con = sql.connect(db)
        cur = con.cursor()
        query_result = cur.execute("SELECT Path_video FROM VIDEO WHERE Path_video LIKE '%england_epl%' OR Path_video LIKE '%france_ligue-1%' OR Path_video LIKE '%europe_uefa-champions-league%'")
        self.training_vids = copy.deepcopy([q[0] for q in query_result.fetchall()])
    
    def _extract_frames(video_path, start_time, end_time, frame_rate):
        
        frames = []
        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Set the starting frame to extract
        cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

        # Calculate the total number of frames to extract
        total_frames = int((end_time - start_time) * frame_rate)

        # Extract the frames
        for i in tqdm(range(total_frames)):
            # Read the next frame
            ret, frame = cap.read()
            if ret:
                # Add the frame to the list
                frames.append(frame)
            else:
                break

            # Set the position to the next frame
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(cap.get(cv2.CAP_PROP_POS_FRAMES)) + int(cap.get(cv2.CAP_PROP_FPS) / frame_rate))

        # Release the video capture
        cap.release()
        return frames
    
    def _process_gameTime(strin):
        # get from '02:44' to 164
        return int(strin[0:2])*60 + int([3:5])
        
    
    def _get_start_times(N=10):
        return np.random.choice(44*60,N)
    
    def __getitem__(self,index):    
        
        #get vid path
        vid = self.training_vids[index]
        # get a list of N start times
        start_times = self._get_start_times()
        
        #2D matrix of extracted images
        X = [_extract_frames(vid,start,start+112,1) for start in start_times]
        
        #path of the json
        json = os.path.join(os.path.dirname(videos[vid]),"Labels-v2.json")
        
        #get list of annotations
        label_list = json['annotations']
        
        #get the game half
        half = int(vid[-9])
        
        for label in label_list:
            if label['gameTime'].contains("{i} - ".format(i=half)):
                time_label = self._process_gameTime(label['gameTime'][4:])
                indices = np.where([(time_label-ti) < 112 and (time_label-ti) >0 for ti in start_times])
                
                
            
            
        
    
     
        

In [17]:
ex = np.random.choice(44*60,10)
ex

array([ 385,  916, 1469, 2261, 1902,  199, 1083, 2272,  453, 1466])

In [21]:
ex[np.where([(1489-ti) < 112 and (1489-ti) > 0  for ti in ex])]

array([1469, 1466])

# MODEL

## 2 Layer MLP

In [7]:
class MLP2(nn.Module):
    def __init__(self,n_features):
        super(MLP2,self).__init__()
        self.conv1 = nn.Conv1d(n_features,MLP_FILTERS_1,MLP_KERNEL_SIZE)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(MLP_FILTERS_1,MLP_FILTERS_2,MLP_KERNEL_SIZE)

    def forward(self,x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out


## UNET ARCHITECTURE

In [30]:
class BottleneckResBlockContract(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P) ->BN ->ReLU -> conv(3,P) -> BN -> ReLU -> conv(1,2P) -> + -> output (of size T,2P)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,2P) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        super(BottleneckResBlockContract,self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,in_features,1)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.conv2 = nn.Conv1d(in_features,in_features,3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(in_features)
        self.conv3 = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)
        self.conv_skip = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z
    
class BottleneckResBlockExpand(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P/2) ->BN ->ReLU -> conv(3,P/2) -> BN -> ReLU -> conv(1,P/2) -> + -> output (of size T,P/2)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,P/2) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        in_features = int(in_features)
        super(BottleneckResBlockExpand,self).__init__()
        self.bn1 = nn.BatchNorm1d(int(in_features))
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)
        self.bn2 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv2 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv3 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),1)
        self.conv_skip = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z

class UpsamplingBlock(nn.Module):
# Takes as input a feature matrix of size (with ncol: nFeatures and nrow: nTimePoints) :
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
# Upsample uses nearest neighbor to output a 2TxP matrix
# a Conv(2,P/2) is then applied with padding = same
    
    def __init__(self,in_feature):
        super(UpsamplingBlock,self).__init__()
        in_feature = int(in_feature)
        self.upsample = nn.Upsample(scale_factor=2,mode=UPSAMPLE_MODE)
        self.conv = nn.Conv1d(in_feature, int(in_feature/2),kernel_size=2, padding=PADDING_METHOD)
    
    def forward(self,x):
        out = self.upsample(x)
        out = self.conv(out)
        return out
    

class Unet1D(nn.Module):
    def __init__(self,in_feature):
        super(Unet1D,self).__init__()

        #LEFT PART
        self.maxpool = nn.MaxPool1d(2)

        self.contr1 = BottleneckResBlockContract(in_feature)
        #self.maxpool
        nb_f = in_feature*2
        self.contr2 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr3 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr4 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        ##############################################

        #DOWN PART
        
        self.contr5 = BottleneckResBlockContract(nb_f)
        nb_f = nb_f*2
        ##############################################

        #RIGHT PART

        self.upsample4 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand4 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample3 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand3 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample2 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand2 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample1 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand1 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample0 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand0 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

    def forward(self,x):
        # X is of shape [nbatch,64,224]

        # LEFT PART
        l0 = x #[P,T]           #Output shape
        a = self.maxpool(l0)    #[P,T/2]
        l1 = self.contr1(a)     #[P*2,T/2]
        b = self.maxpool(l1)    #[P*2,T/4]
        l2 = self.contr2(b)     #[P*4,T/4]
        c = self.maxpool(l2)    #[P*4,T/8]
        l3 = self.contr3(c)     #[P*8,T/8]
        d = self.maxpool(l3)    #[P*8,T/16]
        l4 = self.contr4(d)     #[P*16,T/16]
        e = self.maxpool(l4)    #[P*16, T/32]

        # DOWN PART
        l5 = self.contr5(e)     #[P*32,T/32]

        # RIGHT PART
        r4 = self.upsample4(l5)     #[P*16,T/16]
        t4 = torch.cat((l4,r4),1)   #[P*32,T/16]
        t4 = self.expand4(t4)       #[P*16,T/16]
        r3 = self.upsample3(t4)     #[P*8,T/8]
        t3 = torch.cat((l3,r3),1)   #[P*16,T/8]
        t3 = self.expand3(t3)       #[P*8,T/8]
        r2 = self.upsample2(t3)     #[P*4,T/4]
        t2 = torch.cat((l2,r2),1)   #[P*8,T/4]
        t2 = self.expand2(t2)       #[P*4,T/4]
        r1 = self.upsample1(t2)     #[P*2,T/2]
        t1 = torch.cat((l1,r1),1)   #[P*4,T/2]
        t1 = self.expand1(t1)       #[P*2,T/2]
        r0 = self.upsample0(t1)     #[P,T]
        t0 = torch.concat((l0,r0),1)#[P*2,T]
        t0 = self.expand0(t0)       #[P,T]

        return t0 # P features, T points

## Prediction Head

In [12]:
class PredictionHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


## Temporal Head

In [13]:
class TemporalHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


## Complete models

In [31]:
# Model for prediction
model1 = models.resnet152(weights = models.ResNet152_Weights.IMAGENET1K_V2)
for param in model1.parameters():
    param.requires_grad = False
num_ftrs = model1.fc.in_features
model1.fc = PredictionHead(num_ftrs)
print(model1)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 